In [1]:
pip uninstall openpyxl -Y


Usage:   
  /usr/local/anaconda3/bin/python -m pip uninstall [options] <package> ...
  /usr/local/anaconda3/bin/python -m pip uninstall [options] -r <requirements file> ...

no such option: -Y
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade openpyxl

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --upgrade pandas


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install --upgrade numpy

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 38.0 MB/s eta 0:00:0000:010:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.0
    Uninstalling numpy-2.1.0:
      Successfully uninstalled numpy-2.1.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 2.1.1 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 2.1.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
################################################
################################################technical debt
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

####################### Step 1: Clean the Data

# Load the Excel file (with multiple sheets)
file_path = '/../../merged_file.xlsx'

# Read all sheets into a dictionary of DataFrames
sheets_dict = pd.read_excel(file_path, sheet_name=None)

# Specify the columns to keep for each sheet
columns_to_keep = ['file', 'module_name', 'complexity_rank', 'maintainability_index',
                   'complexity_score', 'effort', 'difficulty', 'bugs', 'vocabulary', 'volume']

# Filter the data for each sheet
for sheet_name, df in sheets_dict.items():
    # Keep only the specified columns
    filtered_df = df[columns_to_keep]
    
    # Drop columns that contain any null values
    sheets_dict[sheet_name] = filtered_df.dropna(axis=1, how='any')

####################### Step 2: Perform TTI and Ti Calculations

# Adjusting the function to correctly apply the thresholds element-wise
def calculate_metric_cost(row, lt, ut):
    metric_costs = {}
    for column in row.index:
        value = row[column]
        lt_value = lt[column]
        ut_value = ut[column]
        if value < lt_value:
            metric_costs[column] = value / lt_value
        elif value > ut_value:
            metric_costs[column] = value / ut_value
        else:
            metric_costs[column] = value / min(lt_value, ut_value)
    return pd.Series(metric_costs)

# Process each sheet for TTI and Ti calculations
for sheet_name, df in sheets_dict.items():
    metric_columns = df.columns[2:]  # Assuming the first two columns are 'file' and 'module_name'
    
    LT = df[metric_columns].quantile(0.25)
    UT = df[metric_columns].quantile(0.75)

    # Apply the function across each row in the dataframe
    tti_df = df[metric_columns].apply(calculate_metric_cost, axis=1, lt=LT, ut=UT)

    # Add the TTI results back to the original dataframe, prefixed with 'TTI_'
    for col in tti_df.columns:
        df[f'TTI_{col}'] = tti_df[col]

    # Calculate the average TTI and add it as a new column
    df['TTI_adjusted'] = tti_df.sum(axis=1) / (4 * tti_df.shape[1])

  ############ Ti Calculation

    TTI_volume_index = df.columns.get_loc('TTI_volume')

# Updated list of columns to consider for the operation
    columns_to_consider = df.columns[9: TTI_volume_index + 1]
    
    # Calculate the average of each column
    averages = df[columns_to_consider].mean()
    print(f"Averages for {sheet_name}:")
    print(averages)

    # Apply the condition: if the value is equal to or above average, set Ti=1, else Ti=0
    for col in columns_to_consider:
        df[f'Ti_{col}'] = df[col].apply(lambda x: 1 if x >= averages[col] else 0)
    
    # Calculate the sum of all Ti_ columns and then find the average
    ti_columns = [col for col in df.columns if col.startswith('Ti_')]
    df['Ti_adjusted'] = df[ti_columns].sum(axis=1) / (4 * len(ti_columns))

    # Add the two columns TTI_adjusted and Ti_adjusted
    df['Total_T_adjusted'] = df['TTI_adjusted'] + df['Ti_adjusted']

####################### Step 3: Save the DataFrame and Color the Headers

# Save the updated dataframes to a new Excel file (with multiple sheets)
output_path_with_ti_average = '/../../filtered_technicalDebt.xlsx'

with pd.ExcelWriter(output_path_with_ti_average, engine='openpyxl') as writer:
    for sheet_name, df in sheets_dict.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

# Load the workbook and apply styling to each sheet
wb = load_workbook(output_path_with_ti_average)

# Define the green, red, and blue fills
green_fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")
red_fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")
blue_fill = PatternFill(start_color="0000FF", end_color="0000FF", fill_type="solid")

# Apply the coloring logic to each sheet
for sheet_name in wb.sheetnames:
    ws = wb[sheet_name]
    
    # Apply the green fill to the titles from 'file' to 'volume'
    for cell in ws[1]:  # First row contains the headers
        if cell.value in ['file', 'module_name', 'complexity_rank', 'maintainability_index',
                          'complexity_score', 'effort', 'difficulty', 'bugs', 'vocabulary', 'volume']:
            cell.fill = green_fill
    
    # Apply the red fill to TTI_ columns' titles and blue fill to Ti_ columns' titles
    for cell in ws[1]:  # First row contains the headers
        if cell.value.startswith('TTI_'):
            cell.fill = red_fill
        elif cell.value.startswith('Ti_'):
            cell.fill = blue_fill

# Save the workbook with the updated styles
wb.save(output_path_with_ti_average)

print(f"Processed and saved Excel file with multiple sheets to {output_path_with_ti_average}")


Averages for Sheet1:
TTI_maintainability_index    1.722146
TTI_complexity_score         2.264662
TTI_effort                   4.220200
TTI_difficulty               1.293426
TTI_bugs                     2.194645
TTI_vocabulary               1.769783
TTI_volume                   2.194645
dtype: float64
Averages for Sheet2:
TTI_maintainability_index    1.030651
TTI_complexity_score         1.883615
TTI_effort                   3.137207
TTI_difficulty               1.557891
TTI_bugs                     2.094004
TTI_vocabulary               1.792366
TTI_volume                   2.094004
dtype: float64
Processed and saved Excel file with multiple sheets to /../../filtered_technicalDebt.xlsx


In [8]:
import pandas as pd
from openpyxl import load_workbook
from sklearn.preprocessing import LabelEncoder
from openpyxl.styles import PatternFill

####################### Step 1: Clean the Data

# Load the Excel file (with multiple sheets)
file_path = '/../../merged_file.xlsx'

# Read all sheets into a dictionary of DataFrames
sheets_dict = pd.read_excel(file_path, sheet_name=None)

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Specify the columns to keep for each sheet
columns_to_keep = ['file', 'module_name', 'loc', 'category', 'lloc', 'sloc', 'multi', 'length', 'blank', 'comments', 'calculated_length']

# Filter the data for each sheet
for sheet_name, df in sheets_dict.items():
    # Convert 'category' column to numeric values (if it exists in the sheet)
    if 'category' in df.columns:
        df['category_numeric'] = label_encoder.fit_transform(df['category'])
    else:
        df['category_numeric'] = None
    
    # Keep only the specified columns
    filtered_df = df[columns_to_keep]
    
    # Convert relevant columns to numeric, coerce errors to NaN
    filtered_df = filtered_df.apply(pd.to_numeric, errors='coerce')
    
    # Drop columns that contain any null values
    filtered_df = filtered_df.dropna(axis=1, how='any')
    
    # Drop rows with NaNs in the remaining columns
    filtered_df = filtered_df.dropna()
    
    sheets_dict[sheet_name] = filtered_df

####################### Step 2: Perform MTI and Mi Calculations

# Adjusting the function to correctly apply the thresholds element-wise
def calculate_metric_cost(row, lt, ut):
    metric_costs = {}
    for column in row.index:
        value = row[column]
        lt_value = lt[column]
        ut_value = ut[column]
        
        # Handle cases where lt_value or ut_value might be zero or NaN
        if pd.isna(lt_value) or pd.isna(ut_value) or lt_value == 0 or ut_value == 0:
            metric_costs[column] = 0  # Assign 0 or any other default value
        else:
            if value < lt_value:
                metric_costs[column] = value / lt_value
            elif value > ut_value:
                metric_costs[column] = value / ut_value
            else:
                metric_costs[column] = value / min(lt_value, ut_value)
    return pd.Series(metric_costs)

# Process each sheet for MTI and Mi calculations
for sheet_name, df in sheets_dict.items():
    metric_columns = df.columns[2:]  # Assuming the first two columns are 'file' and 'module_name'
    
    LT = df[metric_columns].quantile(0.25)
    UT = df[metric_columns].quantile(0.75)

    # Apply the function across each row in the dataframe
    mti_df = df[metric_columns].apply(calculate_metric_cost, axis=1, lt=LT, ut=UT)

    # Add the MTI results back to the original dataframe, prefixed with 'MTI_'
    for col in mti_df.columns:
        df[f'MTI_{col}'] = mti_df[col]

    # Calculate the average MTI and add it as a new column
    df['MTI_adjusted'] = mti_df.sum(axis=1) / (4 * mti_df.shape[1])

    ############ Mi Calculation

    MTI_calculated_length_index = df.columns.get_loc('MTI_calculated_length')

    # Updated list of columns to consider for the operation
    columns_to_consider = df.columns[11: MTI_calculated_length_index + 1]
    
    # Calculate the average of each column
    averages = df[columns_to_consider].mean()
    print(f"Averages for {sheet_name}:")
    print(averages)

    # Apply the condition: if the value is equal to or above average, set Mi=1, else Mi=0
    for col in columns_to_consider:
        df[f'Mi_{col}'] = df[col].apply(lambda x: 1 if x >= averages[col] else 0)
    
    # Calculate the sum of all Mi_ columns and then find the average
    mi_columns = [col for col in df.columns if col.startswith('Mi_')]
    df['Mi_adjusted'] = df[mi_columns].sum(axis=1) / (4 * len(mi_columns))

    # Add the two columns MTI_adjusted and Mi_adjusted
    df['Total_M_adjusted'] = df['MTI_adjusted'] + df['Mi_adjusted']

####################### Step 3: Save the DataFrame and Color the Headers

# Save the updated dataframes to a new Excel file (with multiple sheets)
output_path_with_mi_average = '/../../filtered_Metrics.xlsx'

with pd.ExcelWriter(output_path_with_mi_average, engine='openpyxl') as writer:
    for sheet_name, df in sheets_dict.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

# Load the workbook and apply styling to each sheet
wb = load_workbook(output_path_with_mi_average)

# Define the green, red, and blue fills
green_fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")
red_fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")
blue_fill = PatternFill(start_color="0000FF", end_color="0000FF", fill_type="solid")

# Apply the coloring logic to each sheet
for sheet_name in wb.sheetnames:
    ws = wb[sheet_name]
    
    # Apply the green fill to the titles from 'file' to 'calculated_length'
    for cell in ws[1]:  # First row contains the headers
        if cell.value in ['file', 'module_name', 'loc', 'category', 'lloc', 'sloc', 'multi', 'length', 'blank', 'comments', 'calculated_length']:
            cell.fill = green_fill
    
    # Apply the red fill to MTI_ columns' titles and blue fill to Mi_ columns' titles
    for cell in ws[1]:  # First row contains the headers
        if cell.value.startswith('MTI_'):
            cell.fill = red_fill
        elif cell.value.startswith('Mi_'):
            cell.fill = blue_fill

# Save the workbook with the updated styles
wb.save(output_path_with_mi_average)

print(f"Processed and saved Excel file with multiple sheets to {output_path_with_mi_average}")


Averages for Sheet1:
MTI_blank                1.821411
MTI_comments             1.671049
MTI_calculated_length    2.159159
dtype: float64
Averages for Sheet2:
MTI_blank                1.483580
MTI_comments             2.386457
MTI_calculated_length    2.354788
dtype: float64
Processed and saved Excel file with multiple sheets to /../..//Desktop/../../filtered_Metrics.xlsx


In [11]:
#severity calculation
import pandas as pd
import os

# File paths
metrics_file_path = '/../../filtered_Metrics.xlsx'
TD_file_path = '/../../filtered_technicalDebt.xlsx'
output_combined_path = '/../../combined.xlsx'
output_path_grouped = '/../../SF.xlsx'

# Ensure the directories exist for saving the output files
output_dir_combined = os.path.dirname(output_combined_path)
output_dir_grouped = os.path.dirname(output_path_grouped)
os.makedirs(output_dir_combined, exist_ok=True)
os.makedirs(output_dir_grouped, exist_ok=True)

# Load the necessary columns from the metrics file
metrics_sheets = pd.read_excel(metrics_file_path, sheet_name=None)  # Load all sheets
td_sheets = pd.read_excel(TD_file_path, sheet_name=None)  # Load all sheets

# Define a function to process each sheet
def process_sheet(sheet_name, df_metrics, df_td):
    # Ensure the sheets have the same structure (index alignment)
    df_combined = df_metrics.copy()
    df_combined['Total_T_adjusted'] = df_td['Total_T_adjusted']
    df_combined['Total_adjusted_combined'] = df_combined['Total_M_adjusted'] + df_combined['Total_T_adjusted']
    
    # Define the quartile thresholds
    q1 = df_combined['Total_adjusted_combined'].quantile(0.25)
    q2 = df_combined['Total_adjusted_combined'].quantile(0.50)
    q3 = df_combined['Total_adjusted_combined'].quantile(0.75)
    
    # Function to assign groups based on quartile thresholds
    def assign_group(value):
        if value <= q1:
            return 'Low'
        elif q1 < value <= q2:
            return 'Medium'
        elif q2 < value <= q3:
            return 'High'
        else:
            return 'Severe'
    
    # Apply the function to create a new column 'Risk_Group'
    df_combined['Risk_Group'] = df_combined['Total_adjusted_combined'].apply(assign_group)
    
    return df_combined

# Process each sheet
for sheet_name, df_metrics in metrics_sheets.items():
    if sheet_name in td_sheets:
        df_td = td_sheets[sheet_name]
        # Process and save each sheet
        df_combined = process_sheet(sheet_name, df_metrics, df_td)
        
        # Save the updated DataFrame with the grouped data to a new Excel file
        output_sheet_path = os.path.join(output_dir_grouped, f"{sheet_name}_SF.xlsx")
        df_combined.to_excel(output_sheet_path, index=False)
        print(f"Processed and saved {sheet_name} to {output_sheet_path}")

print("All sheets processed and files saved successfully.")


Processed and saved Sheet1 to /../../SF.xlsx
Processed and saved Sheet2 to/../../SF.xlsx
All sheets processed and files saved successfully.


In [16]:
#####generating SF for training #CSV file generation for training # for now 2 CSV file

import pandas as pd

# File paths
sf_file_path = '/../../SF.xlsx'
merged_file_path = '/../../merged_file.xlsx'
output_risk_csv_path = '/../../training.csv'

# Load the 'Risk_Group' column from the SF file
df_sf = pd.read_excel(sf_file_path, usecols=['Risk_Group'])

# Load the entire merged1 file
df_merged = pd.read_excel(merged_file_path)

# Add the 'Risk_Group' column to the merged DataFrame
df_merged['Risk_Group'] = df_sf['Risk_Group']

# Save the updated DataFrame to a new CSV file
df_merged.to_csv(output_risk_csv_path, index=False)
